In [23]:
from azure.cosmos import CosmosClient, PartitionKey
import json 

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

client = CosmosClient(url, credential=key)

database_name = 'cosmicworks'
container_name = 'products'

database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

with open('cropped_fields/extracted_text.json', 'r') as file:
    item = json.load(file)

item["category"] = "text_data"

container.create_item(body=item, enable_automatic_id_generation=True)

print("Patient data successfully saved to Cosmos DB with auto-generated ID.")


Patient data successfully saved to Cosmos DB with auto-generated ID.


In [33]:
from azure.cosmos import CosmosClient
from cryptography.fernet import Fernet
import json

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

with open("fernet.key", "rb") as f:
    fernet_key = f.read()

cipher = Fernet(fernet_key)

client = CosmosClient(url, credential=key)

database_name = 'cosmicworks'
container_name = 'products'
database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

with open("cropped_fields/extracted_text.json", "r") as file:
    data = json.load(file)

encrypted = cipher.encrypt(json.dumps(data).encode()).decode()

container.create_item(
    body={"category": "text_data", "encrypted_data": encrypted},
    enable_automatic_id_generation=True
)

print("Encrypted and saved new item.")

Encrypted and saved new item.


In [34]:
from azure.cosmos import CosmosClient
from cryptography.fernet import Fernet, InvalidToken
import json

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

with open("fernet.key", "rb") as f:
    fernet_key = f.read()

cipher = Fernet(fernet_key)

client = CosmosClient(url, credential=key)

database_name = 'cosmicworks'
container_name = 'products'

database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

query = "SELECT * FROM c WHERE c.category = 'text_data'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))
    
for item in items:
    encrypted = item.get('encrypted_data')
    if encrypted:
        try:
            decrypted_data = cipher.decrypt(encrypted.encode()).decode()
            print(json.loads(decrypted_data))
        except InvalidToken:
            print(f"Skipped invalid encrypted data for item ID: {item.get('id')}")


{'cin_passport_subscriber': '04026240', 'patient_name': 'Abderrazek Mouhamed', 'subscriber_name': 'Abderazak Braham', 'subscriber_address': 'Ariana'}
{'cin_passport_subscriber': '12345678', 'patient_name': 'Warda Warda', 'subscriber_name': 'Abderazak Braham', 'subscriber_address': 'Sousse'}
{'cin_passport_subscriber': '12345678', 'patient_name': 'Warda Warda', 'subscriber_name': 'Warda Braham', 'subscriber_address': 'Sousse'}
{'cin_passport_subscriber': '98765432', 'patient_name': 'Mohamed Mouhamed', 'subscriber_name': 'Abderazak Braham', 'subscriber_address': 'Sfax'}


## PowerBI

In [36]:
from azure.cosmos import CosmosClient
from cryptography.fernet import Fernet, InvalidToken
import json
import csv

url = "https://insuredpatients.documents.azure.com:443/"
key = "AQFk1movQza5orCTnBNi1ZlOBTZPiGp0gTMSLEZOLXtXxYp4EKlGLMyor7VCZivXJIeZCGOgjq6vACDbqbsyLQ=="

with open("fernet.key", "rb") as f:
    fernet_key = f.read()

cipher = Fernet(fernet_key)

client = CosmosClient(url, credential=key)
database_name = 'cosmicworks'
container_name = 'products'

database = client.get_database_client(database_name)
container = database.get_container_client(container_name)

query = "SELECT * FROM c WHERE c.category = 'text_data'"
items = list(container.query_items(query=query, enable_cross_partition_query=True))

csv_filename = "insuredpatients_data.csv"
with open(csv_filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['cin_passport_subscriber', 'patient_name', 'subscriber_name', 'subscriber_address']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()  
    
    for item in items:
        if 'encrypted_data' in item:
            encrypted_data = item['encrypted_data']
            try:
                decrypted_data = cipher.decrypt(encrypted_data.encode()).decode()
                decrypted_json = json.loads(decrypted_data)

                writer.writerow({
                    'cin_passport_subscriber': decrypted_json.get('cin_passport_subscriber', ''),
                    'patient_name': decrypted_json.get('patient_name', ''),
                    'subscriber_name': decrypted_json.get('subscriber_name', ''),
                    'subscriber_address': decrypted_json.get('subscriber_address', '')
                })
            except InvalidToken:
                print(f"Skipping invalid encrypted data for item ID: {item.get('id')}")
        else:
            print(f"No encrypted data found for item ID: {item.get('id')}")

print(f"Decrypted data saved to {csv_filename}")


Decrypted data saved to insuredpatients_data.csv
